In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import kaggle
import os
import holidays
import tensorflow as tf

2025-02-10 17:44:19.944140: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 17:44:19.953148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739231059.964121   11140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739231059.967496   11140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 17:44:19.978631: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Model Plan:
- Find a api for real time stock data
- Find a api for real time news
- Feature engineer features
- Build Multi-modal model for day trading.
- First branch of model will be Time Series analysis model that will take the real time stock data as a sequence
- Second branch of model will be a text classifier/sentiment analysis model that can take the real time news
- Combine the two branches to make the multi-modal model

# Agent Plan:
- Use same apis to get input data (This is how agent recieves input)
- Use LLM with function calling to helper function and to run the model (Thinking part)
- Have the agent output the result or make the trade itself using the api (The action/goal achieving part)


## Data Collection

In [2]:
# datasets = [
#     "varpit94/apple-stock-data-updated-till-22jun2021",
#     "prajwaldongre/nvidia-corp-share-price-2000-2024",
#     "varpit94/microsoft-stock-data",
#     "guillemservera/amzn-stock-data",
#     "henryshan/meta-platforms-inc-meta",
#     "henryshan/google-stock-price",
#     "guillemservera/tsla-stock-data"
# ]
# for dataset in datasets: 
#     kaggle.api.dataset_download_files(dataset, path='/home/misha/Desktop/data/day_trader/', unzip=True)

In [3]:
df = pd.DataFrame()
for file in os.listdir('/home/misha/Desktop/data/day_trader/'):
    data = pd.read_csv('/home/misha/Desktop/data/day_trader/' + file)
    data['filename'] = file
    data.columns = [col.strip().title() for col in data.columns]
    df = pd.concat([df, data], axis=0, ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53862 entries, 0 to 53861
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            53862 non-null  object 
 1   Open            53862 non-null  float64
 2   High            53862 non-null  float64
 3   Low             53862 non-null  float64
 4   Close           53862 non-null  float64
 5   Volume          53862 non-null  int64  
 6   Raw_Close       10245 non-null  float64
 7   Change_Percent  20486 non-null  float64
 8   Avg_Vol_20D     20414 non-null  float64
 9   Filename        53862 non-null  object 
 10  Adj Close       33372 non-null  float64
 11  Adjusted_Close  10245 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 4.9+ MB


In [5]:
df.drop(['Raw_Close', 'Change_Percent', 'Avg_Vol_20D', 'Adjusted_Close', 'Adj Close'], axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53862 entries, 0 to 53861
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      53862 non-null  object 
 1   Open      53862 non-null  float64
 2   High      53862 non-null  float64
 3   Low       53862 non-null  float64
 4   Close     53862 non-null  float64
 5   Volume    53862 non-null  int64  
 6   Filename  53862 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 2.9+ MB


In [7]:
df.isna().sum()

Date        0
Open        0
High        0
Low         0
Close       0
Volume      0
Filename    0
dtype: int64

In [8]:
filename_to_ticker_map = {
    'tsla_split_adjusted.csv':'TSLA',
    'MSFT.csv':'MSFT',
    'tsla_raw_data.csv':'TSLA',
    'AAPL.csv':'AAPL',
    'META.csv':'META',
    'NVDA.csv':'NVDA',
    'GOOG.csv':'GOOGL',
    'amzn_split_adjusted.csv':'AMZN',
    'amzn_raw_data.csv':'AMZN'
}

In [9]:
df['Ticker'] = df['Filename'].apply(lambda x: filename_to_ticker_map[x])
df.drop('Filename', axis=1, inplace=True)

In [10]:
df['Date'] = pd.to_datetime(df['Date'])

In [11]:
df['Ticker'].value_counts()

Ticker
AMZN     13546
AAPL     10409
MSFT      9083
TSLA      6944
NVDA      6116
GOOGL     4858
META      2906
Name: count, dtype: int64

In [12]:
df.set_index(['Ticker', 'Date'], inplace=True)
df

Open       High        Low      Close     Volume
Ticker Date                                                             
TSLA   2010-06-29    1.26666    1.66666    1.16934    1.59266  281749173
       2010-06-30    1.71934    2.02794    1.55334    1.58866  257915884
       2010-07-01    1.66666    1.72800    1.35134    1.46400  123447940
       2010-07-02    1.53334    1.54000    1.24734    1.28000   77127102
       2010-07-06    1.33334    1.33334    1.05534    1.07400  103189435
...                      ...        ...        ...        ...        ...
AMZN   2024-04-09  187.24000  187.34000  184.20000  185.67000   36546953
       2024-04-10  182.76500  186.26990  182.67000  185.95000   35879152
       2024-04-11  186.74000  189.77000  185.51000  189.05000   40020738
       2024-04-12  187.72000  188.38000  185.08000  186.13000   38608848
       2024-04-15  187.42500  188.69000  183.00000  183.62000   48052395

[53862 rows x 5 columns]

## Feature engineering

The target will be a binary classification. 1 means that the close price will be greater than the open price. 0 means that the close price will be smaller than the open price

In [13]:
## Target Variable
df['Target'] = (df['Close'] > df['Open']).astype(int)

### Other indicators/features

In [14]:
## Time based features
df['Day Of Week'] = df.index.get_level_values('Date').dayofweek
df['Month'] = df.index.get_level_values('Date').month
df['Quarter'] = df.index.get_level_values('Date').quarter

In [15]:
## Holiday Proximity (How many days until the next holiday)

df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'])

# Function to calculate proximity to the nearest holiday
def holiday_metrics(date):
    """
    Calculate both days until next holiday and days to closest holiday.
    
    Args:
        date: datetime object representing the date to check
        
    Returns:
        tuple: (days_until_next, days_to_closest)
            - days_until_next: number of days until the next upcoming holiday
            - days_to_closest: number of days to the nearest holiday in any direction
    """
    # Generate holidays for current and next year
    us_holidays = holidays.US(years=[date.year, date.year + 1])
    holiday_dates = pd.to_datetime(list(us_holidays.keys()))
    
    # For next holiday
    future_holidays = holiday_dates[holiday_dates > date]
    if len(future_holidays) > 0:
        next_holiday = future_holidays.min()
        days_until_next = (next_holiday - date).days
    else:
        days_until_next = np.nan
    
    # For closest holiday
    if len(holiday_dates) > 0:
        days_diff = np.abs((holiday_dates - date).days)
        days_to_closest = np.min(days_diff)
    else:
        days_to_closest = np.nan
        
    return days_until_next, days_to_closest

# Apply the function to each row
df['Days Until Next Holiday'], df['Days To Closest Holiday'] = zip(*df['Date'].apply(holiday_metrics))

# Set the index back to Ticker and Date if needed
df.set_index(['Ticker', 'Date'], inplace=True)

## Data Preperation

Keep in mind that the final day in the series should NOT have the close feature.